In [1]:
from sklearn import preprocessing
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split
import scipy as sp
from scipy import signal
from statsmodels import robust
import glob
import io
import csv
import os
import pandas as pd
import json
import numpy as np
import random
gesture = ["spaghetti"]
path = "trimmed_data_updated/"+gesture[0]+"/"
data = {}
w = "20"
filenames = {"gyro"+ w +"-left2a", "gyro"+ w +"-right2a", "acc"+ w +"-left2a", "acc"+ w +"-right2a", "emg"+ w +"-left2a", "emg"+ w +"-right2a"};

for f in filenames:
    
    data[f] = pd.read_csv(path+f+".csv")

dataFrameDictionary = {}

arms = ["left", "right"]

emgDataFrames = {}
emgDataFrames['right'] = data["emg"+ w +"-right2a"]
emgDataFrames['left'] = data["emg"+ w +"-left2a"]

accDataFrames = {}
accDataFrames['right'] = data["acc"+ w +"-right2a"]
accDataFrames['left'] = data["acc"+ w +"-left2a"]

gyroDataFrames = {}
gyroDataFrames['right'] = data["gyro"+ w +"-right2a"]
gyroDataFrames['left'] = data["gyro"+ w +"-left2a"]

dataFrameDictionary['emg'] = emgDataFrames
dataFrameDictionary['acc'] = accDataFrames
dataFrameDictionary['gyro'] = gyroDataFrames

features ={}
features['left'] = []
features['right'] = []

dfColumns = ["left Variance of Gyro x", "left Variance of Gyro y", "left Variance of Gyro z", 
         "left Mav of Gyro x", "left Mav of Gyro y", "left Mav of Gyro z", "left Rms of Gyro x", "left Rms of Gyro y", 
         "left Rms of Gyro z", "left Sd of Gyro x", "left Sd of Gyro y", "left Sd of Gyro z", "left Mean of Gyro x", 
         "left Mean of Gyro y", "left Mean of Gyro z", "left Entropy of Gyro x", "left Entropy of Gyro y", 
         "left Entropy of Gyro z", "left Kurtosis of Gyro x", "left Kurtosis of Gyro y", "left Kurtosis of Gyro z", 
         "left Skewness of Gyro x", "left Skewness of Gyro y", "left Skewness of Gyro z", "left SMA of Gyro", 
         "left Integration of Gyro x", "left Integration of Gyro y", "left Integration of Gyro z", 
         "right Variance of Gyro x", "right Variance of Gyro y", "right Variance of Gyro z", "right Mav of Gyro x", 
         "right Mav of Gyro y", "right Mav of Gyro z", "right Rms of Gyro x", "right Rms of Gyro y", 
         "right Rms of Gyro z", "right Sd of Gyro x", "right Sd of Gyro y", "right Sd of Gyro z", "right Mean of Gyro x", 
         "right Mean of Gyro y", "right Mean of Gyro z", "right Entropy of Gyro x", "right Entropy of Gyro y", 
         "right Entropy of Gyro z", "right Kurtosis of Gyro x", "right Kurtosis of Gyro y", "right Kurtosis of Gyro z", 
         "right Skewness of Gyro x", "right Skewness of Gyro y", "right Skewness of Gyro z", "right SMA of Gyro", 
         "right Integration of Gyro x", "right Integration of Gyro y", "right Integration of Gyro z", "left Variance of ACC x", 
         "left Variance of ACC y", "left Variance of ACC z", "left Mav of ACC x", "left Mav of ACC y", "left Mav of ACC z", 
         "left Rms of ACC x", "left Rms of ACC y", "left Rms of ACC z", "left Sd of ACC x", "left Sd of ACC y", 
         "left Sd of ACC z", "left Mean of ACC x", "left Mean of ACC y", "left Mean of ACC z", "left Entropy of ACC x", 
         "left Entropy of ACC y", "left Entropy of ACC z", "left Kurtosis of ACC x", "left Kurtosis of ACC y", 
         "left Kurtosis of ACC z", "left Skewness of ACC x", "left Skewness of ACC y", "left Skewness of ACC z", 
         "left SMA of ACC", "left Integration of ACC x", "left Integration of ACC y", "left Integration of ACC z", 
         "right Variance of ACC x", "right Variance of ACC y", "right Variance of ACC z", "right Mav of ACC x", 
         "right Mav of ACC y", "right Mav of ACC z", "right Rms of ACC x", "right Rms of ACC y", "right Rms of ACC z", 
         "right Sd of ACC x", "right Sd of ACC y", "right Sd of ACC z", "right Mean of ACC x", "right Mean of ACC y", 
         "right Mean of ACC z", "right Entropy of ACC x", "right Entropy of ACC y", "right Entropy of ACC z", 
         "right Kurtosis of ACC x", "right Kurtosis of ACC y", "right Kurtosis of ACC z", "right Skewness of ACC x", 
         "right Skewness of ACC y", "right Skewness of ACC z", "right SMA of ACC", "right Integration of ACC x", 
         "right Integration of ACC y", "right Integration of ACC z", "left variance of EMG1", "left variance of EMG2", 
         "left variance of EMG3", "left variance of EMG4", "left variance of EMG5", "left variance of EMG6", 
         "left variance of EMG7", "left variance of EMG8", "left Mav of EMG1", "left Mav of EMG2", "left Mav of EMG3", 
         "left Mav of EMG4", "left Mav of EMG5", "left Mav of EMG6", "left Mav of EMG7", "left Mav of EMG8", "left Rms of EMG1", 
         "left Rms of EMG2", "left Rms of EMG3", "left Rms of EMG4", "left Rms of EMG5", "left Rms of EMG6", "left Rms of EMG7", 
         "left Rms of EMG8", "left Sd of EMG1", "left Sd of EMG2", "left Sd of EMG3", "left Sd of EMG4", "left Sd of EMG5", 
         "left Sd of EMG6", "left Sd of EMG7", "left Sd of EMG8", "left Mean of EMG1", "left Mean of EMG2", "left Mean of EMG3", 
         "left Mean of EMG4", "left Mean of EMG5", "left Mean of EMG6", "left Mean of EMG7", "left Mean of EMG8", 
         "left MedianFreq of EMG1", "left MedianFreq of EMG2", "left MedianFreq of EMG3", "left MedianFreq of EMG4", 
         "left MedianFreq of EMG5", "left MedianFreq of EMG6", "left MedianFreq of EMG7", "left MedianFreq of EMG8", 
         "left MeanFreq of EMG1", "left MeanFreq of EMG2", "left MeanFreq of EMG3", "left MeanFreq of EMG4", 
         "left MeanFreq of EMG5", "left MeanFreq of EMG6", "left MeanFreq of EMG7", "left MeanFreq of EMG8", 
         "right variance of EMG1", "right variance of EMG2", "right variance of EMG3", "right variance of EMG4", 
         "right variance of EMG5", "right variance of EMG6", "right variance of EMG7", "right variance of EMG8", 
         "right Mav of EMG1", "right Mav of EMG2", "right Mav of EMG3", "right Mav of EMG4", "right Mav of EMG5", 
         "right Mav of EMG6", "right Mav of EMG7", "right Mav of EMG8", "right Rms of EMG1", "right Rms of EMG2", 
         "right Rms of EMG3", "right Rms of EMG4", "right Rms of EMG5", "right Rms of EMG6", "right Rms of EMG7", 
         "right Rms of EMG8", "right Sd of EMG1", "right Sd of EMG2", "right Sd of EMG3", "right Sd of EMG4", "right Sd of EMG5", 
         "right Sd of EMG6", "right Sd of EMG7", "right Sd of EMG8", "right Mean of EMG1", "right Mean of EMG2", 
         "right Mean of EMG3", "right Mean of EMG4", "right Mean of EMG5", "right Mean of EMG6", "right Mean of EMG7", 
         "right Mean of EMG8", "right MedianFreq of EMG1", "right MedianFreq of EMG2", "right MedianFreq of EMG3", 
         "right MedianFreq of EMG4", "right MedianFreq of EMG5", "right MedianFreq of EMG6", "right MedianFreq of EMG7", 
         "right MedianFreq of EMG8", "right MeanFreq of EMG1", "right MeanFreq of EMG2", "right MeanFreq of EMG3", 
         "right MeanFreq of EMG4", "right MeanFreq of EMG5", "right MeanFreq of EMG6", "right MeanFreq of EMG7", 
         "right MeanFreq of EMG8"]
print(len(dfColumns))
dfRow = []

224


In [2]:
for a in arms:
    
    df = dataFrameDictionary['gyro'][a]
    x = df.values.astype(float)
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df_normalized = pd.DataFrame(x_scaled)
     
    
    dfRow.append(df_normalized.var()[1])
    dfRow.append(df_normalized.var()[2])
    dfRow.append(df_normalized.var()[3])
    dfRow.append(robust.mad(df_normalized)[1])
    dfRow.append(robust.mad(df_normalized)[2])
    dfRow.append(robust.mad(df_normalized)[3])
    dfRow.append(np.sqrt(np.mean(df_normalized[1].values**2)))
    dfRow.append(np.sqrt(np.mean(df_normalized[2].values**2)))
    dfRow.append(np.sqrt(np.mean(df_normalized[3].values**2)))
    dfRow.append(np.std(df_normalized)[1])
    dfRow.append(np.std(df_normalized)[2])
    dfRow.append(np.std(df_normalized)[3])
    dfRow.append(np.mean(df_normalized)[1])
    dfRow.append(np.mean(df_normalized)[2])
    dfRow.append(np.mean(df_normalized)[3])
    dfRow.append(sp.stats.entropy(df_normalized)[1])
    dfRow.append(sp.stats.entropy(df_normalized)[2])
    dfRow.append(sp.stats.entropy(df_normalized)[3])
    dfRow.append(df_normalized.kurtosis()[1])
    dfRow.append(df_normalized.kurtosis()[2])
    dfRow.append(df_normalized.kurtosis()[3])
    dfRow.append(df_normalized.skew(axis=0)[1])
    dfRow.append(df_normalized.skew(axis=0)[2])
    dfRow.append(df_normalized.skew(axis=0)[3])
    
    length = df_normalized.shape[1]
    sma = 0
    for x in range(0, length):
            # .iloc[x,0] is Gyroessing row, column
        sma += abs(df_normalized.iloc[x,1]) + df_normalized.iloc[x,2] + df_normalized.iloc[x,3]
    sma = sma / length
    dfRow.append(sma)
    dfRow.append(sp.integrate.simps(df_normalized, axis=0)[1])
    dfRow.append(sp.integrate.simps(df_normalized, axis=0)[2])
    dfRow.append(sp.integrate.simps(df_normalized, axis=0)[3])
    
    
    print(a +"Done computing gyro features!!!!!!!")


leftDone computing gyro features!!!!!!!
rightDone computing gyro features!!!!!!!


In [3]:
for a in arms:
    df = dataFrameDictionary['acc'][a]

    x = df.values.astype(float)
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df_normalized = pd.DataFrame(x_scaled)
    
    
    dfRow.append(df_normalized.var()[1])
    dfRow.append(df_normalized.var()[2])
    dfRow.append(df_normalized.var()[3])
    dfRow.append(robust.mad(df_normalized)[1])
    dfRow.append(robust.mad(df_normalized)[2])
    dfRow.append(robust.mad(df_normalized)[3])
    dfRow.append(np.sqrt(np.mean(df_normalized[1].values**2)))
    dfRow.append(np.sqrt(np.mean(df_normalized[2].values**2)))
    dfRow.append(np.sqrt(np.mean(df_normalized[3].values**2)))
    dfRow.append(np.std(df_normalized)[1])
    dfRow.append(np.std(df_normalized)[2])
    dfRow.append(np.std(df_normalized)[3])
    dfRow.append(np.mean(df_normalized)[1])
    dfRow.append(np.mean(df_normalized)[2])
    dfRow.append(np.mean(df_normalized)[3])
    dfRow.append(sp.stats.entropy(df_normalized)[1])
    dfRow.append(sp.stats.entropy(df_normalized)[2])
    dfRow.append(sp.stats.entropy(df_normalized)[3])
    dfRow.append(df_normalized.kurtosis()[1])
    dfRow.append(df_normalized.kurtosis()[2])
    dfRow.append(df_normalized.kurtosis()[3])
    dfRow.append(df_normalized.skew(axis=0)[1])
    dfRow.append(df_normalized.skew(axis=0)[2])
    dfRow.append(df_normalized.skew(axis=0)[3])
    
    length = df_normalized.shape[1]
    sma = 0
    for x in range(0, length):
        sma += abs(df_normalized.iloc[x,1]) + df_normalized.iloc[x,2] + df_normalized.iloc[x,3]
    sma = sma / length
    dfRow.append(sma)
    dfRow.append(sp.integrate.simps(df_normalized, axis=0)[1])
    dfRow.append(sp.integrate.simps(df_normalized, axis=0)[2])
    dfRow.append(sp.integrate.simps(df_normalized, axis=0)[3])
    
    print(a +"Done computing acc features!!!!!!!")

leftDone computing acc features!!!!!!!
rightDone computing acc features!!!!!!!


In [4]:
for a in arms:
    
    df = dataFrameDictionary['emg'][a]
    
    # bandpass filter
    for column in df:
        if "time" not in column:
            emg = np.array(df[column].values)

            emg_correctmean = emg - np.mean(emg)

            # create bandpass filter for EMG
            high = 2/(100/2)
            low = 45/(100/2)
            b, a1 = sp.signal.butter(4, [high,low], btype='bandpass')

            emg_filtered = sp.signal.filtfilt(b, a1, emg_correctmean)

            emg_rectified = abs(emg_filtered)

                # create lowpass filter and apply to rectified signal to get EMG envelope
            sfreq = 100
            low_pass = 10/sfreq
            b2, a2 = sp.signal.butter(4, low_pass, btype='lowpass')
            emg_envelope = sp.signal.filtfilt(b2, a2, emg_rectified)

            df[column] = emg_envelope
    
    filteredDf = df
    
    print("Filtering of " + a + " emg Done!!")
    
   
    x = filteredDf.values.astype(float)
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df_normalized = pd.DataFrame(x_scaled)
    
    dfRow.append(df_normalized.var()[1])
    dfRow.append(df_normalized.var()[2])
    dfRow.append(df_normalized.var()[3])
    dfRow.append(df_normalized.var()[4])
    dfRow.append(df_normalized.var()[5])
    dfRow.append(df_normalized.var()[6])
    dfRow.append(df_normalized.var()[7])
    dfRow.append(df_normalized.var()[8])

    dfRow.append(robust.mad(df_normalized)[1])
    dfRow.append(robust.mad(df_normalized)[2])
    dfRow.append(robust.mad(df_normalized)[3])
    dfRow.append(robust.mad(df_normalized)[4])
    dfRow.append(robust.mad(df_normalized)[5])
    dfRow.append(robust.mad(df_normalized)[6])
    dfRow.append(robust.mad(df_normalized)[7])
    dfRow.append(robust.mad(df_normalized)[8])
    
    dfRow.append(np.sqrt(np.mean(df_normalized[1].values**2)))
    dfRow.append(np.sqrt(np.mean(df_normalized[2].values**2)))
    dfRow.append(np.sqrt(np.mean(df_normalized[3].values**2)))
    dfRow.append(np.sqrt(np.mean(df_normalized[4].values**2)))
    dfRow.append(np.sqrt(np.mean(df_normalized[5].values**2)))
    dfRow.append(np.sqrt(np.mean(df_normalized[6].values**2)))
    dfRow.append(np.sqrt(np.mean(df_normalized[7].values**2)))
    dfRow.append(np.sqrt(np.mean(df_normalized[8].values**2)))
    
    dfRow.append(np.std(df_normalized)[1])
    dfRow.append(np.std(df_normalized)[2])
    dfRow.append(np.std(df_normalized)[3])
    dfRow.append(np.std(df_normalized)[4])
    dfRow.append(np.std(df_normalized)[5])
    dfRow.append(np.std(df_normalized)[6])
    dfRow.append(np.std(df_normalized)[7])
    dfRow.append(np.std(df_normalized)[8])
    
    dfRow.append(np.mean(df_normalized)[1])
    dfRow.append(np.mean(df_normalized)[2])
    dfRow.append(np.mean(df_normalized)[3])
    dfRow.append(np.mean(df_normalized)[4])
    dfRow.append(np.mean(df_normalized)[5])
    dfRow.append(np.mean(df_normalized)[6])
    dfRow.append(np.mean(df_normalized)[7])
    dfRow.append(np.mean(df_normalized)[8])
    
    meanFreq = []
    medFreq = []

    for i in range(1, 9):
        data = df_normalized[i].values
        ps = np.abs(np.fft.fft(data))**2

        freqs = np.fft.fftfreq(data.size, 1/100)

        sumProd = 0
        sumPs = 0

        for f, p in zip(freqs,ps):
            sumProd += f*p
            sumPs += p

        sumProd = sumProd * 100000000
        meanFreq.append(100000000*sumProd/sumPs)
        medFreq.append(sumPs/2)

    dfRow.append(medFreq[0])
    dfRow.append(medFreq[1])
    dfRow.append(medFreq[2])
    dfRow.append(medFreq[3])
    dfRow.append(medFreq[4])
    dfRow.append(medFreq[5])
    dfRow.append(medFreq[6])
    dfRow.append(medFreq[7])
    
    dfRow.append(meanFreq[0])
    dfRow.append(meanFreq[1])
    dfRow.append(meanFreq[2])
    dfRow.append(meanFreq[3])
    dfRow.append(meanFreq[4])
    dfRow.append(meanFreq[5])
    dfRow.append(meanFreq[6])
    dfRow.append(meanFreq[7])
    
    

    print("Done computing emg features of " + a + " arm files")
print(len(dfRow))

Filtering of left emg Done!!
Done computing emg features of left arm files
Filtering of right emg Done!!
Done computing emg features of right arm files
224


In [10]:
print(len(dfRow))

consolidatedFeatures = pd.DataFrame(columns = dfColumns)

consolidatedFeatures.loc[i] = dfRow

224


In [11]:
print(consolidatedFeatures)

   left Variance of Gyro x  left Variance of Gyro y  left Variance of Gyro z  \
8                  0.04246                  0.17587                  0.07046   

   left Mav of Gyro x  left Mav of Gyro y  left Mav of Gyro z  \
8            0.164027            0.102912            0.124831   

   left Rms of Gyro x  left Rms of Gyro y  left Rms of Gyro z  \
8            0.407341            0.565127            0.420781   

   left Sd of Gyro x           ...            right MedianFreq of EMG7  \
8           0.205343           ...                        14175.320951   

   right MedianFreq of EMG8  right MeanFreq of EMG1  right MeanFreq of EMG2  \
8               9780.331992               -0.512295               -1.952392   

   right MeanFreq of EMG3  right MeanFreq of EMG4  right MeanFreq of EMG5  \
8               -0.564021               11.096948                5.409055   

   right MeanFreq of EMG6  right MeanFreq of EMG7  right MeanFreq of EMG8  
8                3.841973             

In [12]:
fileLocation = "WithEmgFilter-NoNormalization.csv"
df = pd.read_csv(fileLocation)

for f in df:
    if(f != "Class"):
        mini = min(df[f])
        maxi = max(df[f])
        
        norm = (consolidatedFeatures[f].iloc[0]-mini) / (maxi - mini) 
        consolidatedFeatures.set_value(8, f, norm)
    


In [13]:
print(consolidatedFeatures)

   left Variance of Gyro x  left Variance of Gyro y  left Variance of Gyro z  \
8                 0.190514                 0.808835                 0.261105   

   left Mav of Gyro x  left Mav of Gyro y  left Mav of Gyro z  \
8            0.242007            0.144369            0.169686   

   left Rms of Gyro x  left Rms of Gyro y  left Rms of Gyro z  \
8            0.310967            0.452335            0.355651   

   left Sd of Gyro x           ...            right MedianFreq of EMG7  \
8           0.346078           ...                            0.197753   

   right MedianFreq of EMG8  right MeanFreq of EMG1  right MeanFreq of EMG2  \
8                  0.093531                     1.0                     1.0   

   right MeanFreq of EMG3  right MeanFreq of EMG4  right MeanFreq of EMG5  \
8                     1.0                     1.0                     1.0   

   right MeanFreq of EMG6  right MeanFreq of EMG7  right MeanFreq of EMG8  
8                     1.0             

In [14]:
import pandas as pd
import numpy as np
import random
import csv
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

fileLocation = "WithEmgFilter.csv"
df = pd.read_csv(fileLocation)

data = df.drop('Class', axis=1)  
classes = df['Class']

X_train, X_test, y_train, y_test = train_test_split(data, classes, test_size = 1)

# print(X_test.columns)
# print(consolidatedFeatures.columns)
    
# print(y_test)
clf = SVC(kernel='linear')
clf.fit(data, classes)  

y_pred = clf.predict(consolidatedFeatures)  


print(y_pred)

['spaghetti']
